In [32]:
import sys
if ".." not in sys.path:
    sys.path.append("..")
import pandas as pd
import numpy as np

from Regression.Linear import Linear, LinearCustom
from Regression.Logistic import Logistic, LogisticCustom
from Classification.bayes import NaiveBayes
from Knn.knn import knn
from Kmeans.Kmeans import KMeans

## Data loading 

In [22]:
with open("data/murders.txt") as f:
    raw_murders_data = pd.read_csv(f, sep=";", index_col=0)

raw_murders_data.head(3)

,Inhabitants,Percent with incomes below $5000,Percent unemployed,"Murders per annum per 1,000,000 inhabitants"
Index,,,,
1,587000,16.5,6.2,11.2
2,643000,20.5,6.4,13.4
3,635000,26.3,9.3,40.7


In [29]:
test_frac = 0.2
div_index = int(test_frac*raw_murders_data.shape[0])
random_order_data = raw_murders_data.values.copy()
np.random.shuffle(random_order_data)

test, train = np.split(random_order_data, [div_index])
x_train, y_train = train[:, :3], train[:, 3]
x_test, y_test = test[:, :3], test[:, 3]